In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

import matplotlib.pyplot as plt

from keras.preprocessing import image
from skimage import io

from sklearn.metrics import confusion_matrix

import seaborn as sn

import json

from bottle import Bottle,route, run, request
import os

# imports necesarios para que funcione la CNN
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization



Using TensorFlow backend.


ESTO ES TEXTO - markdown

In [2]:
# ruta donde se encuentra nuestro dataset.
filename = 'G:/datos/data/fer2013/fer2013.csv'

In [3]:
def checkData():
    # comprobamos la estructura que tiene nuestro dataset.
    df=pd.read_csv(filename)
    print(df)

In [4]:
# funcion para obtener los datos (X=pixels, Y=emotion) y almacenarlos en variables.
def getData():
    # declaramos las variables contenedoras de los pixeles (X) y de las etiquetas (Y).
    X = []
    Y = []
    # evitamos la primera linea que contiene los nombres de las columnas.
    void = True
    for line in open(filename):
        if void:
            void = False
        else:
            # dividimos la linea por la coma de forma que tenemos un array con los datos necesarios.
            fila = line.split(',')
            # apilamos los datos en las variables
            Y.append(int(fila[0]))
            X.append([int(p) for p in fila[1].split()])
    # convertimos a numpy array para tratar los datos.
    # ademas procesamos los pixeles de forma que todos tengan un valor entre 0 y 1 (dividimos entre 255).
    X = np.array(X) / 255.0
    Y = np.array(Y)
    return X, Y

In [5]:
def loadData():
    # llamamos a la funcion para obtener los datos del dataset
    X,Y = getData()
    # identificamos el numero de clases que existen(numero de expresiones faciales)
    num_class = len(set(Y))
    # redimensionamos X para que se adecue a la red neuronal (shape,size,size,gray)
    N,D = X.shape
    X = X.reshape(N, 48, 48, 1)
    # creamos variables de entrenamiento
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
    Y_train = (np.arange(num_class) == Y_train[:, None]).astype(np.float32)
    Y_test = (np.arange(num_class) == Y_test[:, None]).astype(np.float32)
    return X_train,X_test,Y_train,Y_test

In [6]:
def my_model():
    model = Sequential()
    
    model.add(Conv2D(64, (5, 5), input_shape= (48,48,1) ,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    return model

In [7]:
def my_model2():
    INIT_LR = 1e-3
    
    model2 = Sequential()
    model2.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(48,48,1)))
    model2.add(LeakyReLU(alpha=0.1))
    model2.add(MaxPooling2D((2, 2),padding='same'))
    model2.add(Dropout(0.5))

    model2.add(Flatten())
    model2.add(Dense(32, activation='linear'))
    model2.add(LeakyReLU(alpha=0.1))
    model2.add(Dropout(0.5)) 
    model2.add(Dense(7, activation='softmax'))

    model2.summary()

    model2.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])
    
    return model2

In [8]:
def my_matrix():
    X_train, X_test, Y_train, Y_test = loadData()   
    count = 0
    prep_predictions = model.predict_classes(X_test, batch_size=128, verbose=0)
    prep_labels=np.argmax(Y_test, axis=1)
    for i in range (len(prep_predictions)):
        if prep_predictions[i]==prep_labels[i]:
            count = count + 1
    print('Porcentaje de acierto: ',count/len(prep_labels))
    cm = confusion_matrix(prep_labels, prep_predictions)
    %matplotlib inline
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.figure(figsize=(10,8))
    sn.heatmap(cm, annot=True)
    plt.xlabel('PREDICCION')
    plt.ylabel('REAL')
    plt.xticks(y_pos,objects)
    plt.yticks(y_pos,objects)

In [9]:
# Estas son las expresiones disponibles en el dataset
# 0 : angry
# 1 : disgust
# 2 : fear
# 3 : happy
# 4 : sad
# 5 : surprise
# 6 : neutral
expressions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [10]:
# variable que indica si se quiere visualizar los datos
check = True
# variable que indica si existe un modelo guardado ya entrenado
model_saved = True
# variable que indica si se debe imprimir la matriz para analizar resultados
matrix = False

In [11]:
if (check==True):
    checkData()
# ahora sabemos que existen 3 columnas:
#    - emotion: indica que expresion tiene la persona de la foto.
#    - pixels: la foto.
#    - usage: el uso que se le da a dicha foto (Training/PublicTest/PrivateTest).
# NOTA: usaremos todas las fotos del dataset pero posteriormente utilizaremos el 25% para test.

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [12]:
# comprobamos si existe un modelo guardado o no
if(model_saved==False):
    # solicitamos datos
    X_train, X_test, Y_train, Y_test = loadData()
    # limpiamos sesion para intentar evitar entrenamientos largos
    K.tensorflow_backend.clear_session()
    # entrenamos el modelo
    model=my_model()
    # usamos el bakend de keras y seleccionamos el rate de aprendizaje
    K.set_value(model.optimizer.lr,1e-3)
    # hacemos fit con un batch de tamaño 64 y 12 epoch
    h=model.fit(x=X_train,     
            y=Y_train, 
            batch_size=64, 
            epochs=8, 
            verbose=1, 
            validation_data=(X_test,Y_test),
            shuffle=True
            )
    # pasamos el modelo a json para almacenarlo
    model_json = model.to_json()
    with open("expresionsjsonMODEL7.json", "w") as json_file:
        json_file.write(model_json)
    # almacenamos en formato h5
    model.save_weights("expressionsh5MODEL7.h5")
    print("El modelo ha sido generado y almacenado con exito.")
else:
    # el modelo ya existe y no hace falta entrenar la NN, por lo que cargamos el json
    json_file = open('expresionsjsonMODEL2.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # solo necesario si no tenemos un json a partir del que cargar nuestro modelo
    #model=my_model() 
    # cargamos el archivo h5
    model.load_weights("expressionsh5MODEL2.h5")   
    # compilamos el modelo para poder usarlo
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    print("El modelo ha sido cargado.")

El modelo ha sido cargado.


In [13]:
# llamamos a la funcion que pinta el porcentaje de acierto y la matriz
if (matrix==True):
    my_matrix()
# como podemos apreciar la emocion 1 (disgust) no tiene buenos resultados.

In [14]:
def do_prediction(name):
    
    # ESTO COLOCARLO FUERA DE ESTA FUNCION, LA CNN ES INDEPENDIENTE
    img = image.load_img('G:/datos/TFG/photos/'+name)
    img = img.convert('L').resize((48,48))
    #show_img=image.load_img('G:/datos/TFG/photos/'+name)
    
    
    
    
    x = image.img_to_array(img)

    x = np.expand_dims(x, axis = 0)

    x = x / 255

    porcentajes = model.predict(x).tolist()

    # mostramos la foto que nos ha llegado
    #plt.imshow(img)

    jsons = {}
    # recorremos los probabilidades de cada expresion, establecemos como maximo 3 decimales
    a=porcentajes[0]
    for i in range(0,len(a)):
        a[i] = round(a[i],3)
        jsons[expressions[i]]=a[i]
    # devolvemos un json con todas las probabilidades
    return jsons

In [15]:
from bottle import static_file
app = Bottle()
@app.route("/getMood", method="POST")
def do_upload():
    upload = request.files.get("photo")
    save_path = "/datos/TFG/photos/"
    file_path = "{path}/{file}".format(path=save_path, file=upload.filename)

    
    upload.save(file_path)
    
    resultado = do_prediction(upload.filename)
    print(resultado)
    print(type(resultado))
    return resultado

In [16]:
run(app,host='192.168.1.46', port=9004, debug=True)

Bottle v0.12.18 server starting up (using WSGIRefServer())...
Listening on http://192.168.1.46:9004/
Hit Ctrl-C to quit.



{'angry': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happy': 0.998, 'sad': 0.0, 'surprise': 0.0, 'neutral': 0.002}
<class 'dict'>


192.168.1.39 - - [26/Mar/2020 13:38:01] "POST /getMood HTTP/1.1" 200 106


{'angry': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happy': 0.982, 'sad': 0.0, 'surprise': 0.0, 'neutral': 0.018}
<class 'dict'>


192.168.1.39 - - [26/Mar/2020 13:38:09] "POST /getMood HTTP/1.1" 200 106
G:\Programs\Anaconda3\lib\site-packages\bottle.py:3139: ResourceWarning: unclosed <socket.socket fd=3892, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.46', 9004)>
  pass
